docker run --name=deploy_trt2009 --rm -it --runtime nvidia -v ${PWD}:/py -w /py -p 20083:8888 nvcr.io/nvidia/tensorrt:20.09-py3 bash

pip3 install notebook

jupyter notebook --allow-root --ip 0.0.0.0

In [1]:
import sys
import os
import pycuda.driver as cuda
import pycuda.autoinit
import tensorrt as trt
import numpy as np 


In [2]:
trt.__version__

'7.1.3.4'

In [3]:
!pip3 install onnx

You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [4]:
import onnx
import onnx.helper as oh
from onnx import checker
import numpy as np
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit
import sys
import os

AXIS = 1
SHAPE = [1, 3, 4, 5]
# SHAPE = [1, 1, 3, 4, 5]
ONNX_FILE = f"softmax_test{len(SHAPE)}axis{AXIS}.onnx"

# Generate ONNX model
def generate_model(io_shape=SHAPE, axis=AXIS):
    in_tensor = [
        oh.make_tensor_value_info("Input", onnx.TensorProto.FLOAT, io_shape)
    ]
    out_tensor = [
        oh.make_tensor_value_info("Output", onnx.TensorProto.FLOAT, io_shape)
    ]
    nodes = []
    nodes.append(oh.make_node("Softmax", axis=axis, \
        inputs=["Input"], outputs=["Output"]))
    graph = oh.make_graph(nodes, "Test Graph", in_tensor, out_tensor)
    checker.check_graph(graph)
    model = oh.make_model(graph, producer_name="MACNICA", \
        producer_version="0.1")
    checker.check_model(model)
    with open(ONNX_FILE, "wb") as f:
        f.write(model.SerializeToString())

# generate_model(SHAPE, AXIS)
# onnx file already generated in trt6.ipynb
print(ONNX_FILE)

softmax_test4axis1.onnx


In [5]:
!trtexec --onnx=softmax_test4axis1.onnx --saveEngine=softmax_test4axis1_trt7.trt --explicitBatch --fp16 --verbose --dumpOutput

&&&& RUNNING TensorRT.trtexec # trtexec --onnx=softmax_test4axis1.onnx --saveEngine=softmax_test4axis1_trt7.trt --explicitBatch --fp16 --verbose --dumpOutput
[05/24/2021-13:33:55] [I] === Model Options ===
[05/24/2021-13:33:55] [I] Format: ONNX
[05/24/2021-13:33:55] [I] Model: softmax_test4axis1.onnx
[05/24/2021-13:33:55] [I] Output:
[05/24/2021-13:33:55] [I] === Build Options ===
[05/24/2021-13:33:55] [I] Max batch: explicit
[05/24/2021-13:33:55] [I] Workspace: 16 MB
[05/24/2021-13:33:55] [I] minTiming: 1
[05/24/2021-13:33:55] [I] avgTiming: 8
[05/24/2021-13:33:55] [I] Precision: FP32+FP16
[05/24/2021-13:33:55] [I] Calibration: 
[05/24/2021-13:33:55] [I] Safe mode: Disabled
[05/24/2021-13:33:55] [I] Save engine: softmax_test4axis1_trt7.trt
[05/24/2021-13:33:55] [I] Load engine: 
[05/24/2021-13:33:55] [I] Builder Cache: Enabled
[05/24/2021-13:33:55] [I] NVTX verbosity: 0
[05/24/2021-13:33:55] [I] Inputs format: fp32:CHW
[05/24/2021-13:33:55] [I] Outputs format: fp32:CHW
[05/24/2021-13:

[05/24/2021-13:34:04] [V] [TRT] Constructing optimization profile number 0 [1/1].
[05/24/2021-13:34:04] [V] [TRT] --------------- Timing Runner: <reformat> (Reformat)
[05/24/2021-13:34:04] [V] [TRT] Tactic: 1002 time 0.005412
[05/24/2021-13:34:04] [V] [TRT] Tactic: 0 time 0.005008
[05/24/2021-13:34:04] [V] [TRT] Fastest Tactic: 0 Time: 0.005008
[05/24/2021-13:34:04] [V] [TRT] *************** Autotuning format combination: Float(1,5,20,60) -> Float(1,60) ***************
[05/24/2021-13:34:04] [V] [TRT] --------------- Timing Runner: (Unnamed Layer* 0) [Shuffle] (Shuffle)
[05/24/2021-13:34:04] [V] [TRT] Tactic: 0 time 0.004456
[05/24/2021-13:34:04] [V] [TRT] Tactic: 1 time 0.00768
[05/24/2021-13:34:04] [V] [TRT] Fastest Tactic: 0 Time: 0.004456
[05/24/2021-13:34:04] [V] [TRT] *************** Autotuning format combination: Half(1,5,20,60) -> Half(1,60) ***************
[05/24/2021-13:34:04] [V] [TRT] --------------- Timing Runner: (Unnamed Layer* 0) [Shuffle] (Shuffle)
[05/24/2021-13:34:04]

[05/24/2021-13:34:07] [I] Average on 10 runs - GPU latency: 0.00941162 ms - Host latency: 0.0176514 ms (end to end 0.0223267 ms, enqueue 0.00703125 ms)
[05/24/2021-13:34:07] [I] Average on 10 runs - GPU latency: 0.00914307 ms - Host latency: 0.0177246 ms (end to end 0.0238037 ms, enqueue 0.00699463 ms)
[05/24/2021-13:34:07] [I] Average on 10 runs - GPU latency: 0.00946045 ms - Host latency: 0.0175354 ms (end to end 0.0233093 ms, enqueue 0.00750732 ms)
[05/24/2021-13:34:07] [I] Average on 10 runs - GPU latency: 0.0100159 ms - Host latency: 0.0183472 ms (end to end 0.0237793 ms, enqueue 0.00709839 ms)
[05/24/2021-13:34:07] [I] Average on 10 runs - GPU latency: 0.00924683 ms - Host latency: 0.0176819 ms (end to end 0.0226624 ms, enqueue 0.00700073 ms)
[05/24/2021-13:34:07] [I] Average on 10 runs - GPU latency: 0.00928345 ms - Host latency: 0.0175415 ms (end to end 0.0228638 ms, enqueue 0.00706177 ms)
[05/24/2021-13:34:07] [I] Average on 10 runs - GPU latency: 0.0100159 ms - Host latency: 

[05/24/2021-13:34:07] [I] Average on 10 runs - GPU latency: 0.009375 ms - Host latency: 0.0180298 ms (end to end 0.0229858 ms, enqueue 0.00704346 ms)
[05/24/2021-13:34:07] [I] Average on 10 runs - GPU latency: 0.00898437 ms - Host latency: 0.0174072 ms (end to end 0.0223999 ms, enqueue 0.00690918 ms)
[05/24/2021-13:34:07] [I] Average on 10 runs - GPU latency: 0.00981445 ms - Host latency: 0.0179443 ms (end to end 0.023291 ms, enqueue 0.00748291 ms)
[05/24/2021-13:34:07] [I] Average on 10 runs - GPU latency: 0.010144 ms - Host latency: 0.018335 ms (end to end 0.0242432 ms, enqueue 0.00776367 ms)
[05/24/2021-13:34:07] [I] Average on 10 runs - GPU latency: 0.00909424 ms - Host latency: 0.017395 ms (end to end 0.0223877 ms, enqueue 0.00693359 ms)
[05/24/2021-13:34:07] [I] Average on 10 runs - GPU latency: 0.00966797 ms - Host latency: 0.0180054 ms (end to end 0.0230591 ms, enqueue 0.0072998 ms)
[05/24/2021-13:34:07] [I] Average on 10 runs - GPU latency: 0.00963135 ms - Host latency: 0.0181

[05/24/2021-13:34:07] [I] Average on 10 runs - GPU latency: 0.00908203 ms - Host latency: 0.017334 ms (end to end 0.0221436 ms, enqueue 0.0067627 ms)
[05/24/2021-13:34:07] [I] Average on 10 runs - GPU latency: 0.010083 ms - Host latency: 0.0179687 ms (end to end 0.0229736 ms, enqueue 0.00727539 ms)
[05/24/2021-13:34:07] [I] Average on 10 runs - GPU latency: 0.00944824 ms - Host latency: 0.0178955 ms (end to end 0.0226563 ms, enqueue 0.00703125 ms)
[05/24/2021-13:34:07] [I] Average on 10 runs - GPU latency: 0.00881348 ms - Host latency: 0.0172119 ms (end to end 0.0224854 ms, enqueue 0.00678711 ms)
[05/24/2021-13:34:07] [I] Average on 10 runs - GPU latency: 0.00939941 ms - Host latency: 0.0174561 ms (end to end 0.023584 ms, enqueue 0.00727539 ms)
[05/24/2021-13:34:07] [I] Average on 10 runs - GPU latency: 0.00939941 ms - Host latency: 0.0176514 ms (end to end 0.0226318 ms, enqueue 0.00717773 ms)
[05/24/2021-13:34:07] [I] Average on 10 runs - GPU latency: 0.00913086 ms - Host latency: 0.0

In [6]:
TRT_LOGGER = trt.Logger(trt.Logger.VERBOSE)
def load_engine(engine_file_path):
    assert os.path.exists(engine_file_path)
    print(f"Reading engine from file {engine_file_path}")
    with open(engine_file_path,"rb") as f , trt.Runtime(TRT_LOGGER) as runtime:
        return runtime.deserialize_cuda_engine(f.read())

def infer(engine):
    input_image = np.ones((1,3,4,5))
    with engine.create_execution_context() as context:
        context.set_binding_shape(engine.get_binding_index("input"),(1,3,4,5))
        bindings = []
        for binding in engine:
            binding_idx = engine.get_binding_index(binding)
            size = trt.volume(context.get_binding_shape(binding_idx))
            dtype = trt.nptype(engine.get_binding_dtype(binding))
            if engine.binding_is_input(binding):
                input_buffer = np.ascontiguousarray(input_image)
                input_memory = cuda.mem_alloc(input_image.nbytes)
                print("in",list(engine.get_binding_shape(binding)))
                bindings.append(int(input_memory))
            else:
                output_buffer = cuda.pagelocked_empty(size,dtype)
                output_memory = cuda.mem_alloc(output_buffer.nbytes)
                print("out",list(engine.get_binding_shape(binding)))
                bindings.append(int(output_memory))
        stream = cuda.Stream()
        
        cuda.memcpy_htod_async(input_memory,input_buffer,stream)
        context.execute_async_v2(bindings=bindings, stream_handle=stream.handle)
        cuda.memcpy_dtoh_async(output_buffer,output_memory,stream)
        
        stream.synchronize()
    print("engine",engine.max_batch_size)
    print("output_buffer",output_buffer.shape)
    print(output_buffer.reshape(SHAPE))

In [7]:
engine_file = "softmax_test4axis1_trt7.trt"
with load_engine(engine_file) as engine:
    infer(engine)

Reading engine from file softmax_test4axis1_trt7.trt
in [1, 3, 4, 5]
out [1, 3, 4, 5]
engine 1
output_buffer (60,)
[[[[0.00443214 0.02890119 0.00443214 0.02890119 0.00443214]
   [0.02890119 0.00443214 0.02890119 0.00443214 0.02890119]
   [0.00443214 0.02890119 0.00443214 0.02890119 0.00443214]
   [0.02890119 0.00443214 0.02890119 0.00443214 0.02890119]]

  [[0.00443214 0.02890119 0.00443214 0.02890119 0.00443214]
   [0.02890119 0.00443214 0.02890119 0.00443214 0.02890119]
   [0.00443214 0.02890119 0.00443214 0.02890119 0.00443214]
   [0.02890119 0.00443214 0.02890119 0.00443214 0.02890119]]

  [[0.00443214 0.02890119 0.00443214 0.02890119 0.00443214]
   [0.02890119 0.00443214 0.02890119 0.00443214 0.02890119]
   [0.00443214 0.02890119 0.00443214 0.02890119 0.00443214]
   [0.02890119 0.00443214 0.02890119 0.00443214 0.02890119]]]]
